In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Intellectual_Disability"
cohort = "GSE158385"

# Input paths
in_trait_dir = "../../input/GEO/Intellectual_Disability"
in_cohort_dir = "../../input/GEO/Intellectual_Disability/GSE158385"

# Output paths
out_data_file = "../../output/preprocess/Intellectual_Disability/GSE158385.csv"
out_gene_data_file = "../../output/preprocess/Intellectual_Disability/gene_data/GSE158385.csv"
out_clinical_data_file = "../../output/preprocess/Intellectual_Disability/clinical_data/GSE158385.csv"
json_path = "../../output/preprocess/Intellectual_Disability/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Apigenin as a Candidate Prenatal Treatment for Trisomy 21: Effects in Human Amniocytes and the Ts1Cje Mouse Model"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: forebrain'], 1: ['developmental stage: E15'], 2: ['genotype: WT', 'genotype: Ts1Cje'], 3: ['treatment: Powder', 'treatment: Apigenin']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
from typing import Optional, Callable
import os
import json

# Check if gene expression data is available
# From the background information, this appears to be related to trisomy 21 and gene expression
is_gene_available = True

# Define the row indices for trait, age, and gender
# trait_row: Karyotype information (row 2) can be used to determine intellectual disability (trisomy 21)
trait_row = 2  # karyotype information
age_row = None  # No age information available
gender_row = None  # Gender can be inferred from karyotype, but it's not a separate variable for analysis

# Conversion functions
def convert_trait(value: str) -> Optional[int]:
    """Convert karyotype information to binary trait value (1 for T21, 0 for normal)"""
    if not value or ":" not in value:
        return None
    value = value.split(":", 1)[1].strip()
    if "T21" in value:  # Trisomy 21 indicates intellectual disability
        return 1
    elif "2N" in value:  # Normal karyotype
        return 0
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age value to float"""
    # Not used but defined for completeness
    if not value or ":" not in value:
        return None
    value = value.split(":", 1)[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary (0 for female, 1 for male)"""
    # Not used but defined for completeness
    if not value or ":" not in value:
        return None
    value = value.split(":", 1)[1].strip()
    if "female" in value.lower() or "f" == value.lower():
        return 0
    elif "male" in value.lower() or "m" == value.lower():
        return 1
    return None

# Check trait availability
is_trait_available = trait_row is not None

# Save metadata using the validation function
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
# Note: We'll assume the clinical_data is already available as a variable
# from a previous step, rather than loading from a file
if trait_row is not None and 'clinical_data' in locals():
    try:
        # Use the geo_select_clinical_features function to extract features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the selected clinical data
        print("Preview of selected clinical data:")
        print(preview_df(selected_clinical_df))
        
        # Create the output directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical data to a CSV file
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
else:
    if trait_row is not None:
        print("Clinical data not available in memory. Skipping clinical feature extraction.")
    else:
        print("No trait data available. Skipping clinical feature extraction.")


Preview of selected clinical data:
{'GSM4798553': [nan], 'GSM4798554': [nan], 'GSM4798555': [nan], 'GSM4798556': [nan], 'GSM4798557': [nan], 'GSM4798558': [nan], 'GSM4798559': [nan], 'GSM4798560': [nan], 'GSM4798561': [nan], 'GSM4798562': [nan], 'GSM4798563': [nan], 'GSM4798564': [nan], 'GSM4798565': [nan], 'GSM4798566': [nan], 'GSM4798567': [nan], 'GSM4798568': [nan], 'GSM4798569': [nan], 'GSM4798570': [nan], 'GSM4798571': [nan], 'GSM4798572': [nan]}
Clinical data saved to ../../output/preprocess/Intellectual_Disability/clinical_data/GSE158385.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 21225 rows
First 20 gene IDs:
Index(['100008567_at', '100009600_at', '100009609_at', '100009614_at',
       '100012_at', '100017_at', '100019_at', '100033459_at', '100034251_at',
       '100034748_at', '100036520_at', '100036521_at', '100036523_at',
       '100036537_at', '100036768_at', '100037258_at', '100037260_at',
       '100037262_at', '100037278_at', '100037396_at'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Based on my biomedical knowledge, these identifiers (TC01000001.hg.1, etc.) are not standard human gene symbols
# They appear to be Affymetrix transcript cluster IDs from a human gene array
# Standard human gene symbols would be like BRCA1, TP53, etc.
# These IDs need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1647953 rows

Gene annotation preview (first few rows):
{'ID': ['100008567_at', '100009600_at', '100009609_at', '100009614_at', '100012_at'], 'ENTREZ_GENE_ID': ['100008567', '100009600', '100009609', '100009614', '100012'], 'Description': ['predicted gene 14964', 'zinc finger, GATA-like protein 1', 'vomeronasal 2, receptor 65', 'keratin associated protein LOC100009614', 'oogenesin 3']}

Column names in gene annotation data:
['ID', 'ENTREZ_GENE_ID', 'Description']


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Examine the gene identifiers to determine mapping
print("Analyzing gene identifiers for mapping...")

# From the previous output, we have gene annotation with ID, ENTREZ_GENE_ID, and Description
# We'll use the ENTREZ_GENE_ID for mapping since it contains gene identifiers

# Create mapping dataframe using ID and ENTREZ_GENE_ID
print("\nCreating gene mapping dataframe...")
mapping_df = pd.DataFrame({
    'ID': gene_annotation['ID'],
    'Gene': gene_annotation['ENTREZ_GENE_ID']
})

# Keep only rows with valid gene mappings
mapping_df = mapping_df.dropna(subset=['Gene'])
mapping_df = mapping_df[mapping_df['Gene'] != '---']  # Remove any placeholder values
print(f"Created mapping dataframe with {len(mapping_df)} rows")
print("Sample mapping entries:")
print(mapping_df.head())

# 2. Apply gene mapping to convert probe-level measurements to gene expression data
try:
    print("\nApplying gene mapping to expression data...")
    # First, check the overlap between gene expression data IDs and mapping IDs
    overlap_count = sum(gene_data.index.isin(mapping_df['ID']))
    print(f"Overlap between expression data and mapping: {overlap_count} probes out of {len(gene_data.index)}")
    
    if overlap_count > 0:
        # Apply gene mapping to convert probe-level measurements to gene expression data
        gene_data = apply_gene_mapping(gene_data, mapping_df)
        print(f"Successfully mapped to {len(gene_data.index)} genes")
        print("First few gene symbols:")
        print(gene_data.index[:5])
        
        # Optional: Normalize gene symbols to standard forms
        try:
            print("\nNormalizing gene symbols...")
            gene_data = normalize_gene_symbols_in_index(gene_data)
            print(f"After normalization: {len(gene_data.index)} genes")
        except Exception as e:
            print(f"Error normalizing gene symbols: {e}")
            # Continue with unnormalized symbols
    else:
        print("No overlap found between expression data IDs and mapping IDs.")
        print("Using probe IDs directly as gene proxies.")
        # Rename index to Gene for consistency in downstream processing
        gene_data.index.name = 'Gene'
    
    # 3. Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error applying gene mapping: {e}")
    is_gene_available = False


Analyzing gene identifiers for mapping...

Creating gene mapping dataframe...
Created mapping dataframe with 1647953 rows
Sample mapping entries:
             ID       Gene
0  100008567_at  100008567
1  100009600_at  100009600
2  100009609_at  100009609
3  100009614_at  100009614
4     100012_at     100012

Applying gene mapping to expression data...
Overlap between expression data and mapping: 21225 probes out of 21225


Successfully mapped to 0 genes
First few gene symbols:
Index([], dtype='object', name='Gene')

Normalizing gene symbols...
After normalization: 0 genes
Gene expression data saved to ../../output/preprocess/Intellectual_Disability/gene_data/GSE158385.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Use the original gene expression data with probe IDs since normalization gave 0 genes
print("\nHandling gene data...")
try:
    # Load original gene data from previous step if it exists
    if 'gene_data' not in locals() or len(gene_data.index) == 0:
        print("No valid gene symbols after mapping. Using original probe data as gene proxies...")
        # Get original gene expression data again
        gene_data = get_genetic_data(matrix_file)
        # Rename index to Gene for consistency
        gene_data.index.name = 'Gene'
    
    # Save the gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene data saved to: {out_gene_data_file} with {len(gene_data.index)} features")
    is_gene_available = len(gene_data.index) > 0
except Exception as e:
    print(f"Error handling gene data: {e}")
    is_gene_available = False

# 2. Load the clinical data and link with genetic data
print("\nLoading clinical data and linking with genetic data...")
try:
    # Load the clinical data
    clinical_df = pd.read_csv(out_clinical_data_file)
    
    # If clinical_df doesn't have an index column, set the first column as index
    if not clinical_df.index.name and len(clinical_df.columns) > 1:
        clinical_df = clinical_df.set_index(clinical_df.columns[0])
    
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
    print(f"Clinical data columns: {clinical_df.columns}")
    print(f"Clinical data index: {clinical_df.index}")
    
    # Set the appropriate name for the trait in clinical data
    # Since we're working with one trait row from earlier steps
    clinical_df.index = [trait]
    print(f"Updated clinical data index: {clinical_df.index}")
    
    # Ensure we have gene data
    if is_gene_available and not gene_data.empty:
        # Print sample IDs from both datasets for debugging
        print("First few clinical sample IDs:", list(clinical_df.columns)[:5])
        print("First few genetic sample IDs:", list(gene_data.columns)[:5])
        
        # Check and align sample IDs if needed
        common_samples = set(clinical_df.columns).intersection(set(gene_data.columns))
        if len(common_samples) > 0:
            print(f"Found {len(common_samples)} common samples between clinical and genetic data")
            # Keep only common samples
            clinical_subset = clinical_df[list(common_samples)]
            gene_data_subset = gene_data[list(common_samples)]
            
            # Link clinical and genetic data
            linked_data = pd.concat([clinical_subset, gene_data_subset], axis=0).T
            is_trait_available = True
            print(f"Linked data shape: {linked_data.shape}")
            print(f"Linked data columns: {linked_data.columns[:10]}")  # Print first 10 columns
            
            # 3. Handle missing values systematically
            print("\nHandling missing values...")
            try:
                # Make sure the trait column exists in the linked data
                if trait not in linked_data.columns:
                    print(f"Warning: {trait} column not found in linked data. Available columns: {linked_data.columns[:5]}")
                    # If the first column is our trait data, rename it
                    linked_data.rename(columns={linked_data.columns[0]: trait}, inplace=True)
                    print(f"Renamed first column to {trait}")
                
                linked_data = handle_missing_values(linked_data, trait)
                print(f"After handling missing values, data shape: {linked_data.shape}")
                
                # 4. Determine whether the trait and demographic features are biased
                print("\nChecking for bias in features...")
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
            except Exception as e:
                print(f"Error handling missing values: {e}")
                linked_data = pd.DataFrame()
                is_trait_available = False
                is_biased = True
        else:
            print("No common samples found between clinical and genetic data")
            linked_data = pd.DataFrame()
            is_trait_available = False
            is_biased = True
    else:
        print("No valid gene expression data available")
        linked_data = pd.DataFrame()
        is_trait_available = False
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_trait_available = False
    is_biased = True

# 5. Final quality validation
print("\nPerforming final validation...")
note = "Dataset is about trisomy 21 (Down syndrome) which is associated with intellectual disability"

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save linked data if usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Handling gene data...
No valid gene symbols after mapping. Using original probe data as gene proxies...


Gene data saved to: ../../output/preprocess/Intellectual_Disability/gene_data/GSE158385.csv with 21225 features

Loading clinical data and linking with genetic data...
Loaded clinical data with shape: (1, 19)
Clinical data columns: Index(['GSM4798554', 'GSM4798555', 'GSM4798556', 'GSM4798557', 'GSM4798558',
       'GSM4798559', 'GSM4798560', 'GSM4798561', 'GSM4798562', 'GSM4798563',
       'GSM4798564', 'GSM4798565', 'GSM4798566', 'GSM4798567', 'GSM4798568',
       'GSM4798569', 'GSM4798570', 'GSM4798571', 'GSM4798572'],
      dtype='object')
Clinical data index: Index([nan], dtype='float64', name='GSM4798553')
Updated clinical data index: Index(['Intellectual_Disability'], dtype='object')
First few clinical sample IDs: ['GSM4798554', 'GSM4798555', 'GSM4798556', 'GSM4798557', 'GSM4798558']
First few genetic sample IDs: ['GSM4798553', 'GSM4798554', 'GSM4798555', 'GSM4798556', 'GSM4798557']
Found 19 common samples between clinical and genetic data
Linked data shape: (19, 21226)
Linked da